# Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import rdFingerprintGenerator
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
import chembl_structure_pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

[07:18:23] Initializing Normalizer


# Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/KRAS_work_from_insilico.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  452 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

452

# Standardization SDF file for work set

In [5]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  452 molecules


In [6]:
moldf_ws=pd.DataFrame(moldf_ws, columns=['Mol'])
moldf_ws

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x000001E3BED...
1,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
2,<rdkit.Chem.rdchem.Mol object at 0x000001E3BED...
3,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
4,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
...,...
447,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
448,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
449,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
450,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...


# Data entry and curation test set

In [7]:
uploaded_file_ts="datasets/KRAS_test_from_insilico.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  114 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [8]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [9]:
len(y_ts)

114

# Standardization SDF file for test set

In [10]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  114 molecules


In [11]:
moldf_ts=pd.DataFrame(moldf_ts, columns=['Mol'])
moldf_ts

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
1,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
2,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
3,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
4,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
...,...
109,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
110,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
111,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...
112,<rdkit.Chem.rdchem.Mol object at 0x000001E3BEF...


# Calculation Topological Path-Based Fingerprints (RDKit) for work set

In [12]:
import numpy as np
import pandas as pd
from rdkit.Chem import rdFingerprintGenerator

def calcfp(mol, funcFPInfo=dict(minPath=1, maxPath=7, fpSize=2048, useHs=True,
                                branchedPaths=True, useBondOrder=True)):
    """
    RDKit Topological Fingerprint (path-based)
    Returns numpy array for easier DataFrame conversion
    """
    fpgen = rdFingerprintGenerator.GetRDKitFPGenerator(**funcFPInfo)
    fp = fpgen.GetFingerprint(mol)
    # Return numpy array directly - no Series conversion
    return np.asarray(fp)

In [13]:
desc_ws = moldf_ws.Mol.apply(calcfp)
# Convert to regular DataFrame (calcfp returns numpy arrays)
desc_ws = pd.DataFrame(list(desc_ws), index=moldf_ws.index)
desc_ws

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1,1,0,1,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
1,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
2,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
3,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
4,1,1,0,1,1,1,1,1,1,0,...,1,0,0,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1
448,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1
449,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1
450,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1


In [14]:
savetxt('models/TopologicalFingerprint/x_tr_TF.csv', desc_ws, delimiter=',')

In [15]:
y_tr = np.array(y_tr, dtype=np.float32)
len(y_tr)

452

# Calculation Topological Path-Based Fingerprints (RDKit) for test set

In [16]:
desc_ts = moldf_ts.Mol.apply(calcfp)
desc_ts = pd.DataFrame(list(desc_ts), index=desc_ts.index)
desc_ts


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1,1,0,0,0,1,1,0,1,0,...,1,0,0,1,1,0,1,1,1,1
1,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
2,1,1,0,1,0,1,1,1,1,0,...,1,0,1,1,1,0,0,1,1,1
3,1,1,0,0,1,1,1,1,1,0,...,0,1,0,1,1,0,0,1,1,1
4,1,1,1,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,1,1,1,1,0,1,1,0,0,0,...,1,0,0,1,1,0,0,1,1,1
110,1,1,0,1,0,1,1,0,1,0,...,1,0,0,1,1,0,1,1,1,1
111,1,1,0,1,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
112,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1


In [17]:
y_ts = np.array(y_ts, dtype=np.float32)
len(y_ts)

114

# BASELINE

 ## GradientBoostingRegressor model building and validation

In [18]:
seed = 42

In [19]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [20]:
%%time
model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [100,500, 1000]
              }

grid = GridSearchCV(estimator=model, param_grid = parameters, n_jobs=-1, cv = cv)
grid.fit(desc_ws, y_tr, verbose=False)

CPU times: total: 4min
Wall time: 2h 27min 28s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x000001E3BE802630>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [100, 500, 1000],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [21]:
best_CatBR = grid.best_estimator_

In [22]:
grid.best_params_

{'depth': 6, 'iterations': 500, 'learning_rate': 0.1}

In [23]:
params={'verbose': False}

In [24]:
%%time
y_pred_CV_CatBR = cross_val_predict(best_CatBR, desc_ws, y_tr, cv=cv, params=params)

CPU times: total: 11min 33s
Wall time: 1min 2s


In [25]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_CatBR), 2)
Q2_CV

0.6

In [26]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_CatBR)), 2)
RMSE_CV

0.77

In [27]:
y_pred_GBR = best_CatBR.predict(desc_ts)

In [28]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.64

In [29]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.75

save the model to disk

In [30]:
pickle.dump(best_CatBR, open('Models/TopologicalFingerprint/CatBoost_TF.pkl', 'wb'))

load the model from disk

In [31]:
best_CatBR = pickle.load(open('Models/TopologicalFingerprint/CatBoost_TF.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [32]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [33]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7.071068,8.602325,2.828427,9.327379,6.928203,5.567764,7.280110,8.485281,7.416198,15.297059,...,6.082763,6.082763,6.000000,2.236068,6.082763,6.000000,5.830952,6.480741,6.244998,7.280110
2,7.937254,9.643651,3.741657,9.486833,7.211103,7.810250,7.937254,9.643651,11.958261,15.491933,...,6.082763,6.708204,6.164414,6.324555,6.244998,6.480741,8.000000,6.855655,6.244998,9.110434
3,8.366600,9.848858,4.123106,9.746794,7.874008,9.055385,8.000000,10.246951,12.649111,15.556349,...,8.717798,6.708204,6.480741,9.949874,7.348469,6.928203,9.486833,6.855655,6.557439,9.165151
4,8.774964,10.000000,4.242641,9.949874,7.937254,9.110434,8.000000,10.295630,12.961481,15.716234,...,9.327379,7.416198,6.708204,10.535654,7.681146,6.928203,9.746794,8.831761,6.928203,9.899495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,26.286879,27.073973,27.568098,27.147744,26.229754,27.202941,27.982137,27.910571,26.814175,26.115130,...,26.172505,25.980762,26.608269,26.248809,26.381812,25.826343,25.980762,25.709920,26.095977,27.658633
448,26.438608,27.092434,27.586228,27.331301,26.343880,27.239677,28.053520,27.910571,26.888659,26.457513,...,26.514147,26.589472,26.645825,26.324893,26.495283,26.038433,26.229754,25.961510,26.153394,27.676705
449,27.129320,27.276363,27.910571,27.513633,27.147744,27.513633,28.106939,28.425341,27.184554,26.851443,...,26.907248,26.645825,27.221315,27.239677,27.147744,26.210685,26.286879,26.210685,26.514147,27.694765
450,27.276363,27.838822,28.600699,27.784888,27.549955,28.284271,28.142495,28.687977,27.313001,27.838822,...,27.239677,26.981475,27.440845,27.386128,27.294688,26.907248,26.381812,26.758176,27.055499,27.784888


In [34]:
similarity= neighbors_k

In [35]:
Dmean=np.mean(similarity[1,:])

In [36]:
round(Dmean, 2)

6.55

In [37]:
std=np.std(similarity[1,:])

In [38]:
round(std, 2)

3.11

In [39]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

8.11


In [40]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=desc_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [41]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,7.810250,3.741657,7.071068,10.392305,8.062258,13.490738,8.660254,4.472136,1.414214,5.744563,...,9.591663,6.782330,5.830952,2.828427,6.855655,6.480741,4.358899,5.744563,6.324555,5.916080
1,12.569805,4.690416,11.224972,10.535654,8.366600,13.601471,9.110434,4.472136,2.645751,8.306624,...,10.954451,7.483315,6.633250,6.244998,8.000000,6.855655,6.480741,6.244998,6.557439,6.782330
2,12.609520,4.690416,11.313708,10.816654,8.485281,13.638182,9.273618,4.795832,3.316625,9.643651,...,11.000000,11.045361,7.000000,6.324555,8.000000,7.549834,7.416198,7.141428,7.141428,6.782330
3,12.806248,5.385165,11.661904,11.747340,8.544004,13.674794,9.486833,5.000000,3.872983,9.643651,...,12.569805,11.445523,7.071068,8.544004,8.660254,8.888194,7.416198,7.810250,7.937254,7.483315
4,12.806248,5.385165,11.874342,12.041595,8.888194,13.820275,10.000000,5.196152,6.000000,10.148892,...,12.649111,12.409674,8.602325,8.602325,8.774964,9.055385,8.602325,9.000000,8.306624,7.549834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,26.514147,27.367864,26.153394,27.874720,27.404379,27.422618,26.776856,27.477263,27.982137,26.589472,...,26.095977,26.664583,25.748786,25.709920,26.172505,26.400758,25.961510,25.806976,26.381812,26.172505
448,26.532998,27.459060,26.267851,27.910571,27.459060,27.440845,26.814175,27.531800,27.982137,26.776856,...,26.095977,26.702060,26.286879,26.286879,26.551836,26.589472,26.343880,25.980762,26.981475,26.419690
449,26.832816,27.748874,27.110883,27.946377,27.640550,27.477263,27.239677,27.892651,28.017851,27.386128,...,26.115130,26.870058,26.457513,26.419690,27.092434,27.386128,27.000000,26.191602,27.331301,27.331301
450,27.658633,28.513155,27.184554,28.035692,28.372522,28.213472,27.784888,28.583212,28.017851,28.000000,...,26.944387,27.928480,26.832816,26.832816,27.092434,27.531800,27.000000,26.210685,27.622455,27.838822


In [42]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[ 7.81   3.742  7.071 10.392  8.062 13.491  8.66   4.472  1.414  5.745
 11.314  8.544  5.099  8.602  7.681  6.633  7.211 16.823  3.162  5.099
  6.     4.123  6.782  8.246  5.     5.099  9.95   5.099  8.544  3.317
  4.899  2.449  5.477  5.831  7.616  4.69   4.583  6.325  7.616  5.657
  2.449  5.745  4.583  6.164  7.211  2.449  4.359  5.385  5.292  7.
  5.196  3.317  4.472  3.317 11.446  7.55   8.718  6.928 14.731  5.831
  5.916  8.246  4.359  6.403  6.325  6.481  7.     7.     5.385  2.449
  4.123  8.246  9.165  5.657  3.464  5.745  1.732  5.     5.099  6.557
  6.     8.544  6.245  4.     6.245  7.416  5.745  9.     8.     6.403
  6.325  9.381  4.123  2.236  3.464  9.539  4.583  3.     4.243  3.464
  7.141  3.742  4.899  7.     9.592  6.782  5.831  2.828  6.856  6.481
  4.359  5.745  6.325  5.916]


In [43]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True False  True False False  True  True  True False False
  True False  True  True  True False  True  True  True  True  True False
  True  True False  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True False  True False  True
  True False  True  True  True  True  True  True  True  True  True False
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True  True  True False  True  True  True False
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True]


In [44]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD),2))

Coverage =  0.82


In [45]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   2   4   7   8   9  12  14  15  16  18  19  20  21  22  24  25
  27  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  55  57  59  60  62  63  64  65  66  67
  68  69  70  73  74  75  76  77  78  79  80  82  83  84  85  86  88  89
  90  92  93  94  96  97  98  99 100 101 102 103 105 106 107 108 109 110
 111 112 113]


In [46]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [47]:
y_pred_GBR_ad=list(y_pred_GBR)

In [48]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [49]:
len(y_pred_GBR_ad)

93

In [50]:
y_ts_ad=list(y_ts)

In [51]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [52]:
len(y_ts_ad)

93

In [53]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.67

In [54]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.7

# SVM model building and validation

In [55]:
from sklearn.svm import SVR

In [56]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [57]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [58]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [59]:
svm.fit(desc_ws, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [60]:
svm.best_params_
best_svm = svm.best_estimator_

In [61]:
svm.best_params_

{'C': 10, 'gamma': 0.001}

In [62]:
y_pred_CV_svm = cross_val_predict(best_svm, desc_ws, y_tr, cv=cv)

In [63]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.65

In [64]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.72

#  Prediction for test set's molecules

In [65]:
x_ts = np.array(desc_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [66]:
y_pred_svm = best_svm.predict(x_ts)

In [67]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.62

In [68]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.76

save the model to disk

In [69]:
pickle.dump(best_svm, open('Models/TopologicalFingerprint/Topological_FP.pkl', 'wb'))

load the model from disk

In [70]:
best_svm = pickle.load(open('Models/TopologicalFingerprint/Topological_FP.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [71]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [ ]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7.071068,8.602325,2.828427,9.327379,6.928203,5.567764,7.280110,8.485281,7.416198,15.297059,...,6.082763,6.082763,6.000000,2.236068,6.082763,6.000000,5.830952,6.480741,6.244998,7.280110
2,7.937254,9.643651,3.741657,9.486833,7.211103,7.810250,7.937254,9.643651,11.958261,15.491933,...,6.082763,6.708204,6.164414,6.324555,6.244998,6.480741,8.000000,6.855655,6.244998,9.110434
3,8.366600,9.848858,4.123106,9.746794,7.874008,9.055385,8.000000,10.246951,12.649111,15.556349,...,8.717798,6.708204,6.480741,9.949874,7.348469,6.928203,9.486833,6.855655,6.557439,9.165151
4,8.774964,10.000000,4.242641,9.949874,7.937254,9.110434,8.000000,10.295630,12.961481,15.716234,...,9.327379,7.416198,6.708204,10.535654,7.681146,6.928203,9.746794,8.831761,6.928203,9.899495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,26.286879,27.073973,27.568098,27.147744,26.229754,27.202941,27.982137,27.910571,26.814175,26.115130,...,26.172505,25.980762,26.608269,26.248809,26.381812,25.826343,25.980762,25.709920,26.095977,27.658633
448,26.438608,27.092434,27.586228,27.331301,26.343880,27.239677,28.053520,27.910571,26.888659,26.457513,...,26.514147,26.589472,26.645825,26.324893,26.495283,26.038433,26.229754,25.961510,26.153394,27.676705
449,27.129320,27.276363,27.910571,27.513633,27.147744,27.513633,28.106939,28.425341,27.184554,26.851443,...,26.907248,26.645825,27.221315,27.239677,27.147744,26.210685,26.286879,26.210685,26.514147,27.694765
450,27.276363,27.838822,28.600699,27.784888,27.549955,28.284271,28.142495,28.687977,27.313001,27.838822,...,27.239677,26.981475,27.440845,27.386128,27.294688,26.907248,26.381812,26.758176,27.055499,27.784888


In [73]:
similarity= neighbors_k

In [74]:
Dmean=np.mean(similarity[1,:])

In [75]:
round(Dmean, 2)

6.55

In [76]:
std=np.std(similarity[1,:])

In [77]:
round(std, 2)

3.11

In [78]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

8.11


In [79]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [80]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,7.810250,3.741657,7.071068,10.392305,8.062258,13.490738,8.660254,4.472136,1.414214,5.744563,...,9.591663,6.782330,5.830952,2.828427,6.855655,6.480741,4.358899,5.744563,6.324555,5.916080
1,12.569805,4.690416,11.224972,10.535654,8.366600,13.601471,9.110434,4.472136,2.645751,8.306624,...,10.954451,7.483315,6.633250,6.244998,8.000000,6.855655,6.480741,6.244998,6.557439,6.782330
2,12.609520,4.690416,11.313708,10.816654,8.485281,13.638182,9.273618,4.795832,3.316625,9.643651,...,11.000000,11.045361,7.000000,6.324555,8.000000,7.549834,7.416198,7.141428,7.141428,6.782330
3,12.806248,5.385165,11.661904,11.747340,8.544004,13.674794,9.486833,5.000000,3.872983,9.643651,...,12.569805,11.445523,7.071068,8.544004,8.660254,8.888194,7.416198,7.810250,7.937254,7.483315
4,12.806248,5.385165,11.874342,12.041595,8.888194,13.820275,10.000000,5.196152,6.000000,10.148892,...,12.649111,12.409674,8.602325,8.602325,8.774964,9.055385,8.602325,9.000000,8.306624,7.549834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,26.514147,27.367864,26.153394,27.874720,27.404379,27.422618,26.776856,27.477263,27.982137,26.589472,...,26.095977,26.664583,25.748786,25.709920,26.172505,26.400758,25.961510,25.806976,26.381812,26.172505
448,26.532998,27.459060,26.267851,27.910571,27.459060,27.440845,26.814175,27.531800,27.982137,26.776856,...,26.095977,26.702060,26.286879,26.286879,26.551836,26.589472,26.343880,25.980762,26.981475,26.419690
449,26.832816,27.748874,27.110883,27.946377,27.640550,27.477263,27.239677,27.892651,28.017851,27.386128,...,26.115130,26.870058,26.457513,26.419690,27.092434,27.386128,27.000000,26.191602,27.331301,27.331301
450,27.658633,28.513155,27.184554,28.035692,28.372522,28.213472,27.784888,28.583212,28.017851,28.000000,...,26.944387,27.928480,26.832816,26.832816,27.092434,27.531800,27.000000,26.210685,27.622455,27.838822


In [81]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[ 7.81   3.742  7.071 10.392  8.062 13.491  8.66   4.472  1.414  5.745
 11.314  8.544  5.099  8.602  7.681  6.633  7.211 16.823  3.162  5.099
  6.     4.123  6.782  8.246  5.     5.099  9.95   5.099  8.544  3.317
  4.899  2.449  5.477  5.831  7.616  4.69   4.583  6.325  7.616  5.657
  2.449  5.745  4.583  6.164  7.211  2.449  4.359  5.385  5.292  7.
  5.196  3.317  4.472  3.317 11.446  7.55   8.718  6.928 14.731  5.831
  5.916  8.246  4.359  6.403  6.325  6.481  7.     7.     5.385  2.449
  4.123  8.246  9.165  5.657  3.464  5.745  1.732  5.     5.099  6.557
  6.     8.544  6.245  4.     6.245  7.416  5.745  9.     8.     6.403
  6.325  9.381  4.123  2.236  3.464  9.539  4.583  3.     4.243  3.464
  7.141  3.742  4.899  7.     9.592  6.782  5.831  2.828  6.856  6.481
  4.359  5.745  6.325  5.916]


In [82]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True False  True False False  True  True  True False False
  True False  True  True  True False  True  True  True  True  True False
  True  True False  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True False  True False  True
  True False  True  True  True  True  True  True  True  True  True False
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True  True  True False  True  True  True False
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True]


In [83]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.82


In [84]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   2   4   7   8   9  12  14  15  16  18  19  20  21  22  24  25
  27  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  55  57  59  60  62  63  64  65  66  67
  68  69  70  73  74  75  76  77  78  79  80  82  83  84  85  86  88  89
  90  92  93  94  96  97  98  99 100 101 102 103 105 106 107 108 109 110
 111 112 113]


In [85]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [86]:
y_pred_svm_ad=list(y_pred_svm)

In [87]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [88]:
len(y_pred_svm_ad)

93

In [89]:
y_ts_ad=list(y_ts)

In [90]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [91]:
len(y_ts_ad)

93

In [92]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.62

In [93]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.75

# Multi-layer Perceptron regressor

In [94]:
from sklearn.neural_network import MLPRegressor

In [95]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [96]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [97]:
m.fit(desc_ws, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(400, 300, 200, 100),
                                                (100, 100, 100), (10, 10, 10),
                                                (50,)],
                         'max_iter': [1000, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [98]:
best_MLPR = m.best_estimator_

In [99]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, desc_ws, y_tr, cv=cv)

In [100]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.54

In [101]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.82

# Prediction for test set's molecules

In [102]:
y_pred_MLPR = best_MLPR.predict(desc_ts)

In [103]:
Q2_TS = round(r2_score(y_ts, y_pred_MLPR), 2)
Q2_TS

0.58

In [104]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_MLPR)), 2)
RMSE_TS

0.8

save the model to disk

In [105]:
pickle.dump(best_MLPR, open('models/TopologicalFingerprint/MLPR_TF.pkl', 'wb'))

#  Estimating applicability domain. Method - Euclidian distances, K=1

In [ ]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [107]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7.071068,8.602325,2.828427,9.327379,6.928203,5.567764,7.280110,8.485281,7.416198,15.297059,...,6.082763,6.082763,6.000000,2.236068,6.082763,6.000000,5.830952,6.480741,6.244998,7.280110
2,7.937254,9.643651,3.741657,9.486833,7.211103,7.810250,7.937254,9.643651,11.958261,15.491933,...,6.082763,6.708204,6.164414,6.324555,6.244998,6.480741,8.000000,6.855655,6.244998,9.110434
3,8.366600,9.848858,4.123106,9.746794,7.874008,9.055385,8.000000,10.246951,12.649111,15.556349,...,8.717798,6.708204,6.480741,9.949874,7.348469,6.928203,9.486833,6.855655,6.557439,9.165151
4,8.774964,10.000000,4.242641,9.949874,7.937254,9.110434,8.000000,10.295630,12.961481,15.716234,...,9.327379,7.416198,6.708204,10.535654,7.681146,6.928203,9.746794,8.831761,6.928203,9.899495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,26.286879,27.073973,27.568098,27.147744,26.229754,27.202941,27.982137,27.910571,26.814175,26.115130,...,26.172505,25.980762,26.608269,26.248809,26.381812,25.826343,25.980762,25.709920,26.095977,27.658633
448,26.438608,27.092434,27.586228,27.331301,26.343880,27.239677,28.053520,27.910571,26.888659,26.457513,...,26.514147,26.589472,26.645825,26.324893,26.495283,26.038433,26.229754,25.961510,26.153394,27.676705
449,27.129320,27.276363,27.910571,27.513633,27.147744,27.513633,28.106939,28.425341,27.184554,26.851443,...,26.907248,26.645825,27.221315,27.239677,27.147744,26.210685,26.286879,26.210685,26.514147,27.694765
450,27.276363,27.838822,28.600699,27.784888,27.549955,28.284271,28.142495,28.687977,27.313001,27.838822,...,27.239677,26.981475,27.440845,27.386128,27.294688,26.907248,26.381812,26.758176,27.055499,27.784888


In [108]:
similarity= neighbors_k

In [109]:
Dmean=np.mean(similarity[1,:])

In [110]:
round(Dmean, 2)

6.55

In [111]:
std=np.std(similarity[1,:])

In [112]:
round(std, 2)

3.11

In [113]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

8.11


In [114]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=desc_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [115]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,7.810250,3.741657,7.071068,10.392305,8.062258,13.490738,8.660254,4.472136,1.414214,5.744563,...,9.591663,6.782330,5.830952,2.828427,6.855655,6.480741,4.358899,5.744563,6.324555,5.916080
1,12.569805,4.690416,11.224972,10.535654,8.366600,13.601471,9.110434,4.472136,2.645751,8.306624,...,10.954451,7.483315,6.633250,6.244998,8.000000,6.855655,6.480741,6.244998,6.557439,6.782330
2,12.609520,4.690416,11.313708,10.816654,8.485281,13.638182,9.273618,4.795832,3.316625,9.643651,...,11.000000,11.045361,7.000000,6.324555,8.000000,7.549834,7.416198,7.141428,7.141428,6.782330
3,12.806248,5.385165,11.661904,11.747340,8.544004,13.674794,9.486833,5.000000,3.872983,9.643651,...,12.569805,11.445523,7.071068,8.544004,8.660254,8.888194,7.416198,7.810250,7.937254,7.483315
4,12.806248,5.385165,11.874342,12.041595,8.888194,13.820275,10.000000,5.196152,6.000000,10.148892,...,12.649111,12.409674,8.602325,8.602325,8.774964,9.055385,8.602325,9.000000,8.306624,7.549834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,26.514147,27.367864,26.153394,27.874720,27.404379,27.422618,26.776856,27.477263,27.982137,26.589472,...,26.095977,26.664583,25.748786,25.709920,26.172505,26.400758,25.961510,25.806976,26.381812,26.172505
448,26.532998,27.459060,26.267851,27.910571,27.459060,27.440845,26.814175,27.531800,27.982137,26.776856,...,26.095977,26.702060,26.286879,26.286879,26.551836,26.589472,26.343880,25.980762,26.981475,26.419690
449,26.832816,27.748874,27.110883,27.946377,27.640550,27.477263,27.239677,27.892651,28.017851,27.386128,...,26.115130,26.870058,26.457513,26.419690,27.092434,27.386128,27.000000,26.191602,27.331301,27.331301
450,27.658633,28.513155,27.184554,28.035692,28.372522,28.213472,27.784888,28.583212,28.017851,28.000000,...,26.944387,27.928480,26.832816,26.832816,27.092434,27.531800,27.000000,26.210685,27.622455,27.838822


In [116]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[ 7.81   3.742  7.071 10.392  8.062 13.491  8.66   4.472  1.414  5.745
 11.314  8.544  5.099  8.602  7.681  6.633  7.211 16.823  3.162  5.099
  6.     4.123  6.782  8.246  5.     5.099  9.95   5.099  8.544  3.317
  4.899  2.449  5.477  5.831  7.616  4.69   4.583  6.325  7.616  5.657
  2.449  5.745  4.583  6.164  7.211  2.449  4.359  5.385  5.292  7.
  5.196  3.317  4.472  3.317 11.446  7.55   8.718  6.928 14.731  5.831
  5.916  8.246  4.359  6.403  6.325  6.481  7.     7.     5.385  2.449
  4.123  8.246  9.165  5.657  3.464  5.745  1.732  5.     5.099  6.557
  6.     8.544  6.245  4.     6.245  7.416  5.745  9.     8.     6.403
  6.325  9.381  4.123  2.236  3.464  9.539  4.583  3.     4.243  3.464
  7.141  3.742  4.899  7.     9.592  6.782  5.831  2.828  6.856  6.481
  4.359  5.745  6.325  5.916]


In [117]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True False  True False False  True  True  True False False
  True False  True  True  True False  True  True  True  True  True False
  True  True False  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True False  True False  True
  True False  True  True  True  True  True  True  True  True  True False
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True  True  True False  True  True  True False
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True]


In [118]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.82


In [119]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   2   4   7   8   9  12  14  15  16  18  19  20  21  22  24  25
  27  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  55  57  59  60  62  63  64  65  66  67
  68  69  70  73  74  75  76  77  78  79  80  82  83  84  85  86  88  89
  90  92  93  94  96  97  98  99 100 101 102 103 105 106 107 108 109 110
 111 112 113]


In [120]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [121]:
y_pred_MLPR_ad=list(y_pred_MLPR)

In [122]:
y_pred_MLPR_ad[:] = [x for i,x in enumerate(y_pred_MLPR_ad) if i not in out_Ad]

In [123]:
len(y_pred_MLPR_ad)

93

In [124]:
y_ts_ad=list(y_ts)

In [125]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [126]:
len(y_ts_ad)

93

In [127]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_MLPR_ad), 2)
Q2_TS

0.56

In [128]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_MLPR_ad)), 2)
RMSE_TS

0.81